In [31]:

import csv
import glob
import shutil
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

# Create a folder with format (11-November 2018) in the directory 
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')
dir_name = 'S://TANF Reports//!2019 TANF Reports//'
os.chdir(dir_name)


folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join('S://TANF Reports//!2019 TANF Reports', folder_name )


for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  


# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))

        x = filename.replace("-", " ") + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)
    
    
# Load Excel
sanc_count = pd.read_excel('Z_Sanctioned Adults By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')
caseload_count = pd.read_excel('Z_Caseload Report' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx') 

# sanction file - Drop columns 
sanction_count = sanc_count.drop(['Casehead Name','SSN', 'Name','State Count', 'Federal Count'], axis='columns')

# Dataframe to be used in excel tab
df_sanction_count = sanction_count.groupby('Local Office').agg({'Case Number': pd.Series.nunique}).reset_index()
df_sanction_count = lo_sanction_count.rename(index=str, columns={'Case Number': 'Sanction Count'})

# caseload - Remove Child Only Cases
caseload_count = caseload_count[caseload_count.Type != 'Child Only']

# caseload - Create Program column based on +ve or -ve case number
caseload_count['Program'] = caseload_count['Case Number'].apply(lambda x: 'TEA' if x >= 0 else 'Work Pays')

# Dataframe to be used in excel tab
df_case_count = caseload_count.groupby(['Local Office', 'Case Manager','Program']).agg({'Case Number': pd.Series.nunique}).reset_index()
#df_case_count = caseload_count.rename(index = str,  columns={'Case Number': 'Caseload Count'})

# caseload aggrigate
cl_agg = caseload_count.groupby('Local Office').agg({'Case Manager': pd.Series.nunique,'Case Number': pd.Series.nunique }).reset_index()
cl_agg = cl_agg.rename(index = str, columns={'Case Manager': 'Number of WFS', 'Case Number': 'Caseload Count'})

# Merge sanction and caseload files
agg_frame = cl_agg.merge(lo_sanction_count, on='Local Office')

# Calculations
agg_frame['Average Cases per WFS'] = agg_frame['Caseload Count'].div(agg_frame['Number of WFS']).round(1)
agg_frame['Aggregate Percentage Sanction'] = (100*(agg_frame['Sanction Count'])).div(agg_frame['Caseload Count']).round(1)

# Reorder columns
agg_frame = agg_frame[['Local Office','Sanction Count', 'Caseload Count', 'Number of WFS', 'Average Cases per WFS', 'Aggregate Percentage Sanction']]

# Stats
# Creating variables for dataframe
max_caseload = agg_frame.loc[(agg_frame['Caseload Count'].idxmax())].to_frame().T.reset_index()
min_caseload = agg_frame.loc[(agg_frame['Caseload Count'].idxmin())].to_frame().T.reset_index()
max_caseload_wfs = agg_frame.loc[(agg_frame['Average Cases per WFS'].idxmax())].to_frame().T.reset_index()
min_caseload_wfs = agg_frame.loc[(agg_frame['Average Cases per WFS'].idxmin())].to_frame().T.reset_index()
max_per_san = agg_frame.loc[(agg_frame['Aggregate Percentage Sanction'].idxmax())].to_frame().T.reset_index()
min_per_san = agg_frame.loc[(agg_frame['Aggregate Percentage Sanction'].idxmin())].to_frame().T.reset_index()

# Local Office Column
lo_max_caseload = max_caseload.loc[0,'Local Office']
lo_min_caseload = min_caseload.loc[0,'Local Office']
lo_max_caseload_wfs = max_caseload_wfs.loc[0,'Local Office']
lo_min_caseload_wfs = min_caseload_wfs.loc[0,'Local Office']
lo_max_per_san = max_per_san.loc[0,'Local Office']
lo_min_per_san = min_per_san.loc[0,'Local Office']

# Number column
num_max_caseload = max_caseload.loc[0,'Caseload Count']
num_min_caseload = min_caseload.loc[0,'Caseload Count']
num_max_caseload_wfs = max_caseload_wfs.loc[0,'Average Cases per WFS']
num_min_caseload_wfs = min_caseload_wfs.loc[0,'Average Cases per WFS']
num_max_per_san = max_per_san.loc[0,'Aggregate Percentage Sanction']
num_min_per_san = min_per_san.loc[0,'Aggregate Percentage Sanction']

# DataFrame
stat_frame = pd.DataFrame({'Summary_Matrix': ['LO with Maximum Caseload',
                                              'LO with Minimum Caseload',
                                             'LO with Maximum Caseload, WFS Ratio',
                                             'LO with Minimum Caseload, WFS Ratio',
                                             'LO with Maximum Sanction Percentage',
                                             'LO with Minimum Sanction Percentage'],
                           'Local_Office':[lo_max_caseload,
                                           lo_min_caseload,
                                           lo_max_caseload_wfs,
                                           lo_min_caseload_wfs,
                                           lo_max_per_san,
                                           lo_min_per_san],
                           'Number': [num_max_caseload,
                                      num_min_caseload,
                                      num_max_caseload_wfs,
                                      num_min_caseload_wfs,
                                      num_max_per_san,
                                      num_min_per_san]},
                         columns = ['Summary_Matrix','Local_Office','Number'])

# Save statistics in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx 
file_name = 'Caseload Breakdown by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')

# Save dataframe in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx
if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book
    
df_sanction_count.to_excel(writer, sheet_name= str_prev_month + '_sanction', index=False)
df_case_count.to_excel(writer, sheet_name= str_prev_month + '_caseload', index=False)
agg_frame.to_excel(writer, sheet_name= str_prev_month + '_aggrigate', index=False)
stat_frame.to_excel(writer, sheet_name= str_prev_month + '_stats', index=False)
writer.save()
writer.close()